## Auto Regressive for Feature Extraction by Lags of time series 
 

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
%matplotlib inline
import mne
mne.utils.use_log_level('error')
from scipy import signal

In [2]:
import glob

path = r'Preprocessed_Data'
all_files = glob.glob(path + "/*_Non-VR.fif")

li, s = [],[]
cols = []
sub = 1
col = ['Features']
for filename in all_files:
    raw =  mne.io.read_raw_fif(filename, preload=True, verbose='error');
    
    if 'CH 4' in raw.info['ch_names']:
        raw.pick_channels(["CH 4"])
        
        picks = mne.pick_types(raw.info, eeg=True, exclude=[])
        interval = 5
        start = 60
        stop = start + interval
        while(stop <= 120):
            start1, stop1 = raw.time_as_index([start, stop])
            data, times = raw[picks[:], start1:stop1]
            #egment = pd.Series(data[0]).shift(17).dropna()
        
            #d = pd.Series(raw.get_data()[0]).shift(17).dropna()
        
            freq, psd = signal.welch(data[0], 128, nperseg=256) #nperseg is 2*sf
            
            psd_data = pd.Series(psd[:41])
            
            li.append(psd_data)
            
            for i in range(len(psd_data)):
                s.append(sub)
                
            
            
            
            start += interval
            stop += interval

            cols.append(s)
    sub += 1
    
s = pd.Series(s)      
frame = pd.concat(li, ignore_index=True, sort=False)

df = pd.DataFrame([frame,s])


In [3]:
df1 = df.T
df1.columns = ['Features','Subjects']
df1

,Features,Subjects
0,0.041116,1.0
1,0.027843,1.0
2,0.007832,1.0
3,0.001364,1.0
4,0.072791,1.0
5,0.814337,1.0
6,4.044754,1.0
7,9.848842,1.0
8,17.734405,1.0
9,15.124915,1.0


In [4]:
df1.shape

(14268, 2)

### Support Vectore Machine (SVM)

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [6]:
X = df1.drop(['Subjects'], axis=1)
y = df1['Subjects']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [7]:
model = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')

In [8]:
model.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
prediction = model.predict(X_test)

In [10]:
print(classification_report(y_test, prediction))

              precision    recall  f1-score   support

         1.0       0.00      0.00      0.00       159
         2.0       0.09      0.18      0.12       137
         3.0       0.00      0.00      0.00       153
         4.0       0.00      0.00      0.00       155
         5.0       0.15      0.99      0.26       132
         6.0       0.00      0.00      0.00       148
         7.0       0.00      0.00      0.00       146
         8.0       0.00      0.00      0.00       165
         9.0       0.45      0.03      0.06       143
        10.0       0.10      0.17      0.13       136
        11.0       0.07      0.34      0.12       152
        12.0       0.00      0.00      0.00       157
        13.0       0.00      0.00      0.00       151
        14.0       0.17      0.05      0.08       153
        15.0       0.06      0.31      0.10       143
        16.0       0.00      0.00      0.00       150
        17.0       0.10      0.09      0.09       147
        18.0       0.13    

c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sonal\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [40]:
set(y_test) - set(prediction)

{9.0, 19.0}